In [2]:
import os, sys, time
import xarray as xr
import pandas as pd
import datetime as dt


In [3]:
datadir = "/p/scratch/deepacf/video_prediction_shared_folder/preprocessedData/T2monthly"

datafile = "1970-1999_t2m.nc"

datafile= os.path.join(datadir, datafile)



In [4]:
with xr.open_dataset(datafile) as dfile:
    t2m_all = dfile["var167"]
    coords = t2m_all.coords

In [5]:
ntimes = len(coords["time"])

t2m_all = t2m_all.chunk({"time": ntimes, "lat":10, "lon":10})

In [6]:
# define a function with the hourly calculation:
def hour_mean(x):
     return x.groupby('time.hour').mean('time')

time0 = time.time()
t2m_hourly = t2m_all.groupby("time.month").apply(hour_mean)

print("Registering averaging took {0:.2f}".format(time.time()-time0))

#print(t2m_hourly.values)

print("Performing averaging took {0:.2f}".format(time.time()-time0))



/p/software/hdfml/stages/2020/software/Jupyter/2020.2.6-gcccoremkl-9.3.0-2020.2.254-Python-3.8.5/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/p/software/hdfml/stages/2020/software/Jupyter/2020.2.6-gcccoremkl-9.3.0-2020.2.254-Python-3.8.5/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    

Registering averaging took 40.18
Performing averaging took 40.18


In [ ]:
t2m_hourly = t2m_hourly.compute()

In [ ]:
print(t2m_hourly)

In [ ]:
from dask.diagnostics import ProgressBar

delayed_obj = t2m_hourly.to_netcdf(os.path.join(datadir, "climatology_t2m_1970_1999.nc"), compute=False)
with ProgressBar():
    results = delayed_obj.compute()